In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import datetime
from time import sleep

from lyricsgenius import Genius
from collections import Counter

import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import pprint
from spotipy.oauth2 import SpotifyClientCredentials

from requests.exceptions import ReadTimeout

In [2]:
CLIENT_ID = '7f8b2e3adcbf4c24a6091bbaf44013de'
CLIENT_SECRET = '71e10a4bb2534a2cb97e19b794ebc38f'


AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [3]:
auth_manager = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=10, retries=10)

In [4]:
genius = Genius('in424DYZRH1oa--Iyzce_Ue6uLYNqt1Q6-Yq20PUSYl-PjvMNuuaX0YBiwcSFt9_')

In [5]:
song_parts_regex = r"\[Intro.*?\]|\[Verse [0-9].*?\]|\[Refrain.*?\]|\[Pre-Chorus.*?\]|\[Chorus.*?\]|\[Post-Chorus.*?\]|\[Hooks.*?\]|\[Riffs/Basslines.*?\]|\[Scratches.*?\]|\[Bridge.*?\]|\[Interlude.*?\]|\[Break.*?\]|\[Skit.*?\]|\[Collision.*?\]|\[Instrumental or Solo.*?\]|\[Ad lib.*?\]|\[Segue.*?\]|\[Outro.*?\]"

In [6]:
def scrape_lyrics(url):
    """
    input (str): url for a Genius lyrics page
    output (tuple list): returns list of song parts and lyrics tuples
    """

    if type(url) != str or len(url) <= 3:
        return []

    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics_result_set = html.find_all('div', attrs={'data-lyrics-container': 'true'})
    lyrics = [lyrics_result.get_text(separator=' ') for lyrics_result in lyrics_result_set]
    lyrics = ' '.join(lyrics)
   
    try:
        parts_iter = re.finditer(song_parts_regex, lyrics)

        song_parts = []
        song_part_spans = []
        for part_object in parts_iter:
            song_parts.append(part_object.group())
            song_part_spans.append(part_object.span())

        lyric_spans = []
        # # If there are lyrics before the first song part, add that range of indices
        # try:
        #     print(song_part_spans[0][0])
        # except:
        #     print("An exception occurred", song_part_spans)
        # first_song_part_start_index = song_part_spans[0][0]
        # if first_song_part_start_index != 0:
        #     song_parts.insert(0, '[Start]')
        #     lyric_spans.append((0, first_song_part_start_index))

        for index in range(len(song_part_spans) - 1):
            start_lyric_span = song_part_spans[index][1]
            end_lyric_span = song_part_spans[index + 1][0]
            lyric_spans.append((start_lyric_span, end_lyric_span))

        # Add the lyrics after the final song part
        lyric_spans.append((song_part_spans[-1][1], len(lyrics)))

        # Get lyrics without song part tags
        span_to_lyrics = lambda span: lyrics[span[0]:span[1]].lower()
        lyrics_without_song_parts = [span_to_lyrics(span).strip() for span in lyric_spans]

        # Create lyrics list
        lyrics = list(zip(song_parts, lyrics_without_song_parts))

        return lyrics
        
    except:
        return []


def extract_lyrics_text(lyrics):
        text = []
        if lyrics:
            # print()
            # print(lyrics)
            # print()
            # print(list(zip(*lyrics)))
            # print()
            # print(list(zip(*lyrics))[1])
            # print()
            # print(' '.join(list(zip(*lyrics))[1]))
            # print()
            # print(list(' '.join(list(zip(*lyrics))[1])))
            return ' '.join(list(zip(*lyrics))[1])

In [72]:
df_charts = pd.read_csv('data/charts.csv')
df_charts.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [77]:
def create_df_billboard(start=0, limit=5, chunk=1000):
    df_charts = pd.read_csv('data/charts.csv')
    song_artist_array = df_charts[['song', 'artist']].to_numpy()
    # print(song_artist_array[:limit])
    # print([song_info for (song_info, _) in Counter(map(tuple, song_artist_array)).most_common()[:limit]])
    songs, artists = list(zip(*[song_info for (song_info, _) in Counter(map(tuple, song_artist_array)).most_common()[start:limit]]))
    print(len(songs))

    urls = []
    peak_ranks = []
    weeks_on_boards = []
    last_weeks = []
    ranks = []
    lyrics_temp = []
    lyrics_text_temp = []
    genres = []
    decades = []
    features = []
    releases = []
    durations = []
    tempos = []
    timesigs = []

    for indx, (song, artist) in enumerate(zip(songs, artists)):
        # print('check1')
        # Spotify/genre work
        q = "artist:" + artist + " track:" + song

        try:
            results = sp.search(q=q, type='track')
        except ReadTimeout:
            print('Spotify track timed out... trying one more time...')
            try:
                results = sp.search(q=q, type='track')
            except ReadTimeout:
                print('Spotify track timed out twice.')
                results = {'tracks' : {'items' : []}}
                # results['tracks']['items'] = []
        # results = sp.search(q=q, type='track')
        # print('check2')
        if len(results['tracks']['items']) > 0:
            
            aux_bool = True
            # artist genre
            try:
                artist_uri = results['tracks']['items'][0]['artists'][0]['uri']
            except: 
                aux_bool = False
            # artist_spot = sp.artist(artist_uri)
            if aux_bool:
                try:
                    # print('check3')
                    artist_spot = sp.artist(artist_uri)
                    # print('check4')
                    genre = artist_spot['genres']
                except ReadTimeout:
                    print('Spotify artist timed out... trying one more time...')
                    try:
                        artist_spot = sp.artist(artist_uri)
                        genre = artist_spot['genres']
                    except ReadTimeout:
                        print('Spotify artist timed out twice.')
                        genre = None
                # genre = artist_spot['genres']
                genres.append(genre)

                # release and decade
                release = results['tracks']['items'][0]['album']['release_date']
                releases.append(release)
                decade = ((int(release[:4]))//10) * 10
                decades.append(decade)

                # song features
                spot_id = results['tracks']['items'][0]['id']
                try:
                    # print('check5')
                    feature = sp.audio_features([spot_id])[0]
                    # print('check6')
                    features.append(feature)
                    if feature:
                            # song duration
                            durations.append(feature['duration_ms'])
                            # song tempo
                            tempos.append(feature['tempo'])
                            # song time signature
                            timesigs.append(feature['time_signature'])
                    else:
                        # song duration
                        durations.append(None)
                        # song tempo
                        tempos.append(None)
                        # song time signature
                        timesigs.append(None)
                except ReadTimeout:
                    print('Spotify features timed out... trying one more time...')
                    try:
                        feature = sp.audio_features([spot_id])[0]
                        features.append(feature)
                        if feature:
                            # song duration
                            durations.append(feature['duration_ms'])
                            # song tempo
                            tempos.append(feature['tempo'])
                            # song time signature
                            timesigs.append(feature['time_signature'])
                        else:
                            # song duration
                            durations.append(None)
                            # song tempo
                            tempos.append(None)
                            # song time signature
                            timesigs.append(None)
                    except ReadTimeout:
                        print('Spotify features timed out twice.')
                        features.append(None)
                        # song duration
                        durations.append(None)
                        # song tempo
                        tempos.append(None)
                        # song time signature
                        timesigs.append(None)
            else:
                genres.append(None)
                releases.append(None)
                decades.append(None)
                features.append(None)
                durations.append(None)
                tempos.append(None)
                timesigs.append(None)
                

        else:
            genres.append(None)
            releases.append(None)
            decades.append(None)
            features.append(None)
            durations.append(None)
            tempos.append(None)
            timesigs.append(None)

        # Genius/lyric work
        start = datetime.datetime.now()
        stop = start + datetime.timedelta(seconds = 5)
        while True:
            if datetime.datetime.now() > stop:
                urls.append(None)
                break
            try:
                urls.append(genius.search_song(song, artist).url)
                break
            except:
                pass
        match_rows = df_charts.loc[(df_charts['song'] == song) & (df_charts['artist'] == artist)]
        peak_rank = {}
        weeks_on_board = {}
        last_week = {}
        rank = {}
        for index, row in match_rows.iterrows():
            rank[row['date']] = row['rank']
            peak_rank[row['date']] = row['peak-rank']
            weeks_on_board[row['date']] = row['weeks-on-board']
            last_week[row['date']] = row['last-week']
        ranks.append(rank)
        peak_ranks.append(peak_rank)
        weeks_on_boards.append(weeks_on_board)
        last_weeks.append(last_week)

        if ((indx) % chunk) == 0 and indx != 0:
            if indx == chunk:
                lyrics_temp = lyrics_temp + list(map(scrape_lyrics, urls[0:]))
                lyrics_text_temp = lyrics_text_temp + list(map(extract_lyrics_text, lyrics_temp))
                # lyrics_temp.append(list(map(scrape_lyrics, urls[0:])))
                # lyrics_text_temp.append(list(map(extract_lyrics_text, lyrics_temp)))
            else:
                lyrics_temp = lyrics_temp + list(map(scrape_lyrics, urls[(indx - chunk + 1):]))
                # lyrics_temp.append(list(map(scrape_lyrics, urls[(indx - chunk + 1):])))
                lyrics_text_temp = lyrics_text_temp + list(map(extract_lyrics_text, lyrics_temp[(indx - chunk + 1):]))
                # lyrics_text_temp.append(list(map(extract_lyrics_text, lyrics_temp[(indx - chunk + 1):])))

            # print(len(songs))
            # print(len(artists))
            # print(len(urls))
            # print(len(lyrics_temp))
            # print(len(lyrics_text_temp))
            # print(len(decades))
            # print(len(timesigs))
            df_billboard_temp = pd.DataFrame({
            'song': songs[:(indx + 1)],
            'artist': artists[:(indx + 1)],
            'url': urls,
            'lyrics': lyrics_temp,
            'lyrics_text': lyrics_text_temp,
            'rank': ranks,
            'peak_rank': peak_ranks,
            'weeks_on_board': weeks_on_boards,
            'last_week': last_weeks,
            'genre': genres,
            'decade': decades,
            'release_date': releases,
            'song_features': features,
            'duration_ms': durations,
            'tempo': tempos,
            'time_signature': timesigs,
            })

            df_billboard_temp.to_csv(('data_scrape/chart_with_lyrics_genres_running_11' + '.csv'), index=False)
            print('wrote: ' + 'chart_with_lyrics_genres_' + str(indx) + '.csv')
        
        sleep(0.6)

    lyrics = list(map(scrape_lyrics, urls))
    lyrics_text = list(map(extract_lyrics_text, lyrics))

    df_billboard = pd.DataFrame({
        'song': songs,
        'artist': artists,
        'url': urls,
        'lyrics': lyrics_temp,
        'lyrics_text': lyrics_text_temp,
        'rank': ranks,
        'peak_rank': peak_ranks,
        'weeks_on_board': weeks_on_boards,
        'last_week': last_weeks,
        'genre': genres,
        'decade': decades,
        'release_date': releases,
        'song_features': features,
        'duration_ms': durations,
        'tempo': tempos,
        'time_signature': timesigs,
    })

    return df_billboard
    

In [78]:
df_billboard = create_df_billboard(start=27799, limit=29681, chunk=10)

1882
Searching for "Yacht Club" by Lil Yachty Featuring Juice WRLD...
Done.
Searching for "Make It Back" by Juice WRLD...
Done.
Searching for "Kiss And Make Up" by Dua Lipa & BLACKPINK...
Searching for "Get Dripped" by Lil Yachty Featuring Playboi Carti...
Done.
Searching for "Flip The Switch" by Quavo Featuring Drake...
Done.
Searching for "Pass Out" by Quavo Featuring 21 Savage...
Done.
Searching for "Biggest Alley Oop" by Quavo...
Done.
Searching for "Rerun" by Quavo Featuring Travis Scott...
Done.
Searching for "Huncho Dreams" by Quavo...
Done.
Searching for "Lose It" by Quavo Featuring Lil Baby...
Done.
Searching for "Off White VLONE" by Lil Baby & Gunna Featuring Lil Durk & NAV...
Done.
wrote: chart_with_lyrics_genres_10.csv
Searching for "Business Is Business" by Lil Baby & Gunna...
Done.
Searching for "Is That Alright?" by Lady Gaga...
Done.
Searching for "Belly" by Lil Baby & Gunna...
Done.
Searching for "My Blood" by twenty one pilots...
Done.
Searching for "Maybe It's Time" 

ValueError: All arrays must be of the same length

In [ ]:
df_charts = pd.read_csv('data/charts.csv')
song_artist_array = df_charts[['song', 'artist']].to_numpy()
# print(song_artist_array[:limit])
# print([song_info for (song_info, _) in Counter(map(tuple, song_artist_array)).most_common()[:limit]])
songs, artists = list(zip(*[song_info for (song_info, _) in Counter(map(tuple, song_artist_array)).most_common()]))

df_new = pd.DataFrame({'song' : songs})
df_new.to_csv(('test.csv'), index=False)

In [7]:
df_charts_final = pd.read_csv('data_scrape/chart_with_lyrics_genres_running.csv')
df_charts_final.head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (10,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,

,song,artist,url,lyrics,lyrics_text,rank,peak_rank,weeks_on_board,last_week,genre,...,Unnamed: 1678,Unnamed: 1679,Unnamed: 1680,Unnamed: 1681,Unnamed: 1682,Unnamed: 1683,Unnamed: 1684,Unnamed: 1685,Unnamed: 1686,Unnamed: 1687
0,Blinding Lights,The Weeknd,NaN,[],NaN,"{'2021-09-04': 20, '2021-08-28': 21, '2021-08-...","{'2021-09-04': 1, '2021-08-28': 1, '2021-08-21...","{'2021-09-04': 90, '2021-08-28': 89, '2021-08-...","{'2021-09-04': 21.0, '2021-08-28': 18.0, '2021...","['canadian contemporary r&b', 'canadian pop', ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Radioactive,Imagine Dragons,https://genius.com/Imagine-dragons-radioactive...,"[('[Intro]', 'whoa, oh-oh whoa, oh-oh whoa, oh...","whoa, oh-oh whoa, oh-oh whoa, oh-oh whoa i'm w...","{'2014-05-10': 49, '2014-05-03': 48, '2014-04-...","{'2014-05-10': 3, '2014-05-03': 3, '2014-04-26...","{'2014-05-10': 87, '2014-05-03': 86, '2014-04-...","{'2014-05-10': 48.0, '2014-05-03': 42.0, '2014...","['modern rock', 'rock']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sail,AWOLNATION,https://genius.com/Awolnation-sail-lyrics,"[('[Intro]', 'sail'), ('[Verse 1]', 'this is h...",sail this is how i show my love i made it in m...,"{'2014-03-22': 45, '2014-03-15': 41, '2014-03-...","{'2014-03-22': 17, '2014-03-15': 17, '2014-03-...","{'2014-03-22': 79, '2014-03-15': 78, '2014-03-...","{'2014-03-22': 41.0, '2014-03-15': 41.0, '2014...","['la indie', 'modern alternative rock', 'moder...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,I'm Yours,Jason Mraz,https://genius.com/Jason-mraz-im-yours-lyrics,"[('[Verse 1]', ""well you done done me in; you ...",well you done done me in; you bet i felt it i ...,"{'2009-10-10': 48, '2009-10-03': 49, '2009-09-...","{'2009-10-10': 6, '2009-10-03': 6, '2009-09-26...","{'2009-10-10': 76, '2009-10-03': 75, '2009-09-...","{'2009-10-10': 49.0, '2009-10-03': 38.0, '2009...","['acoustic pop', 'neo mellow', 'pop', 'pop rock']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,How Do I Live,LeAnn Rimes,https://genius.com/Leann-rimes-how-do-i-live-l...,"[('[Verse 1]', 'how do i get through one night...",how do i get through one night without you? if...,"{'1998-10-10': 45, '1998-10-03': 48, '1998-09-...","{'1998-10-10': 2, '1998-10-03': 2, '1998-09-26...","{'1998-10-10': 69, '1998-10-03': 68, '1998-09-...","{'1998-10-10': 48.0, '1998-10-03': 43.0, '1998...","['country', 'country dawn', 'dance pop']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
splitter = df_charts.shape[0] // 4

-3


In [13]:
df_charts_1 = df_charts.iloc[0:splitter]
df_charts_2 = df_charts.iloc[splitter:splitter*2]
df_charts_3 = df_charts.iloc[splitter*2:splitter*3]
df_charts_4 = df_charts.iloc[splitter*3:]

In [14]:
df_charts_1.to_csv(('data_scrape/chart_with_lyrics_genres_split_1' + '.csv'), index=False)
df_charts_2.to_csv(('data_scrape/chart_with_lyrics_genres_split_2' + '.csv'), index=False)
df_charts_3.to_csv(('data_scrape/chart_with_lyrics_genres_split_3' + '.csv'), index=False)
df_charts_4.to_csv(('data_scrape/chart_with_lyrics_genres_split_4' + '.csv'), index=False)

In [26]:
df_charts_final.loc[df_charts_final['genre'].str.contains('rock', case=False, na=False)]

,song,artist,url,lyrics,lyrics_text,rank,peak_rank,weeks_on_board,last_week,genre,...,Unnamed: 1678,Unnamed: 1679,Unnamed: 1680,Unnamed: 1681,Unnamed: 1682,Unnamed: 1683,Unnamed: 1684,Unnamed: 1685,Unnamed: 1686,Unnamed: 1687


In [28]:
for decade in range(1950, 2021, 10):
    df_rock = df_charts_final.loc[df_charts_final['genre'].str.contains('rock', case=False, na=False)]
    df_rock2 = df_rock.loc[df_rock['decade'] == decade]
    df_pop = df_charts_final.loc[df_charts_final['genre'].str.contains('pop', case=False, na=False)]
    df_pop2 = df_pop.loc[df_pop['decade'] == decade]
    df_rap = df_charts_final.loc[df_charts_final['genre'].str.contains('rap', case=False, na=False)]
    df_rap2 = df_rap.loc[df_rap['decade'] == decade]
    df_rock2.to_csv(('data_scrape/chart_' + str(decade) + '_rock' + '.csv'), index=False)
    df_pop2.to_csv(('data_scrape/chart_' + str(decade) + '_pop' + '.csv'), index=False)
    df_rap2.to_csv(('data_scrape/chart_' + str(decade) + '_rap' + '.csv'), index=False)
